In [14]:
import re
import os
import glob
import math
import numpy as np
from tqdm import tqdm
from scipy.misc import imread

In [15]:
# !pip install Pillow
# !pip install scipy==1.1.0

In [16]:
"""
predictions_path = '../../Datasets/Huawei/DATASET_INPAINTING/train_gt'

for filename in tqdm(os.listdir(predictions_path)):
    if '_gt' in filename:
        try:
            os.rename(
                os.path.join(predictions_path, filename),
                os.path.join(predictions_path, re.sub('_gt','',filename))
            )
        except:
            print('Problems with: ', filename)
"""

100%|██████████| 801/801 [00:00<00:00, 552844.74it/s]


In [17]:
predictions_path = '../../Datasets/Huawei/DATASET_INPAINTING/test'
gt_path = '../../Datasets/Huawei/DATASET_INPAINTING/test_gt'

In [38]:
def get_list_of_files(folder):
    return glob.glob(os.path.join(folder,'*.png')) + glob.glob(os.path.join(folder,'*.jpg'))


def psnr(y, x):
    mse = np.mean((x.astype(np.float64) - y.astype(np.float64)) ** 2)
    if mse == 0:
        return math.inf
    return 20 * math.log10(255.0 / math.sqrt(mse))


def l1_loss(y, x):
    return np.sum(np.abs(x / 256. - y / 256.)) / 3.  


def compute_metrics(predictions_dir, gt_dir):
    metrics = []
    
    filenames = [f.split('/')[-1] for f in get_list_of_files(predictions_dir)]
    filenames_gt = [f.split('/')[-1] for f in get_list_of_files(gt_dir)]
    assert set(filenames) == set(filenames_gt)
    
    for i, filename in enumerate(filenames):
        prediction_image = imread(os.path.join(predictions_dir, filename))
        gt_image = imread(os.path.join(gt_dir, filename))

        metrics.append({
            'filename': filename,
            'psnr': psnr(prediction_image, gt_image),
            'l1': l1_loss(prediction_image, gt_image)
        })
    
    mean_psnr = round(np.mean([f['psnr'] for f in metrics]), 4)
    mean_l1 = round(np.mean([f['l1'] for f in metrics]), 4)
    
    print(
        "PSNR: %.4f" % mean_psnr,
        "L1: %.4f" % mean_l1,
    )

    return mean_psnr, mean_l1, metrics

In [39]:
mean_psnr, mean_l1, metrics = compute_metrics(predictions_dir, gt_dir)

/Users/anatolijpotapov/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/Users/anatolijpotapov/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


PSNR: 17.8172 L1: 11608.4084
